# Working with large catalogs

Large astronomical surveys contain a massive volume of data. Billion object, multi-terabyte sized catalogs are challenging to store and manipulate because they demand state-of-the-art hardware. Processing them is expensive, both in terms of runtime and memory consumption, and performing it in a single machine has become impractical. LSDB is a solution that enables scalable algorithm execution. It handles loading, querying, filtering and crossmatching astronomical data (of HiPSCat format) in a distributed environment. 

In this tutorial, we will demonstrate how to:

1. Setup a Dask client for distributed processing
2. Load an object catalog with a set of desired columns
3. Select data from regions of the sky
4. Preview catalog data

In [ ]:
import lsdb

## Installing dependencies

To load catalogs from the web using the HTTP protocol we'll need to install __aiohttp__.

In [ ]:
!pip install aiohttp --quiet

## Creating a Dask client

Dask is a framework that allows us to take advantage of distributed computing capabilities. 

With Dask, the operations defined in a workflow (e.g. this notebook) are added to a task graph which optimizes their order of execution. The operations are not immediately computed, that's for us to decide. As soon as we trigger the computations, Dask distributes the workload across its multiple workers and tasks are run efficiently in parallel. Usually the later we kick off the computations the better.

Dask creates a client by default, if we do not instantiate one. If we do, we may, among others:

- Specify the number of workers and the memory limit for each of them.
- Adjust the address for the dashboard to profile the operations while they run (by default it serves on port _8787_).

For additional information please refer to https://distributed.dask.org/en/latest/client.html.

In [ ]:
from dask.distributed import Client

client = Client(n_workers=4, memory_limit="auto")
client

## Loading a catalog

We will load a small 5 degree radius cone from the `ZTF DR14` object catalog. 

Catalogs represent tabular data and are internally subdivided into partitions based on their positions in the sky. When processing a catalog, each worker is expected to be able to load a single partition at a time into memory for processing. Therefore, when loading a catalog, it's crucial to __specify the subset of columns__ we need for our science pipeline. Failure to specify these columns results in loading the entire partition table, which not only increases the usage of worker memory but also impacts runtime performance significantly.

In [ ]:
surveys_path = "https://epyc.astro.washington.edu/~lincc-frameworks/other_degree_surveys"

In [ ]:
ztf_object_path = f"{surveys_path}/ztf/ztf_object"
ztf_object = lsdb.read_hipscat(ztf_object_path, columns=["ps1_objid", "ra", "dec"])
ztf_object

The catalog has been loaded lazily, we can see its metadata but no actual data is there yet. We will be defining more operations in this notebook. Only when we call `compute()` on the resulting catalog are operations executed, i.e. data is loaded from disk into memory for processing.

## Selecting a region of the sky

We may use 3 types of spatial filters (cone, polygon and box) to select a portion of the sky. 

Filtering consists of two main steps:

- A __coarse__ stage, in which we find what pixels cover our desired region in the sky. These may overlap with the region and only be partially contained within the region boundaries. This means that some data points inside that pixel may fall outside of the region.

- A __fine__ stage, where we filter the data points from each pixel to make sure they fall within the specified region.

The `fine` parameter allows us to specify whether or not we desire to run the fine stage, for each search. It brings some overhead, so if your intention is to get a rough estimate of the data points for a region, you may disable it. It is always executed by default.

```
catalog.box(..., fine=False)
catalog.cone_search(..., fine=False)
catalog.polygon_search(..., fine=False)
```

Throughout this notebook we will use the Catalog's `plot_pixels` method to display the HEALPix of each resulting catalog as filters are applied.

In [ ]:
ztf_object.plot_pixels("ZTF_DR14 - pixel map")

### Cone search

A cone search is defined by center `(ra, dec)`, in degrees, and radius `r`, in arcseconds.

In [ ]:
ztf_object_cone = ztf_object.cone_search(ra=-60.3, dec=20.5, radius_arcsec=1 * 3600)
ztf_object_cone

In [ ]:
ztf_object_cone.plot_pixels("ZTF_DR14 - cone pixel map")

### Polygon search

A polygon search is defined by convex polygon with vertices `[(ra1, dec1), (ra2, dec2)...]`, in degrees.

In [ ]:
vertices = [(-60.5, 15.1), (-62.5, 18.5), (-65.2, 15.3), (-64.2, 12.1)]
ztf_object_polygon = ztf_object.polygon_search(vertices)
ztf_object_polygon

In [ ]:
ztf_object_polygon.plot_pixels("ZTF_DR14 - polygon pixel map")

### Box search

A box search can be defined by:

- Right ascension band `(ra1, ra2)`
- Declination band `(dec1, dec2)`
- Both right ascension and declination bands `[(ra1, ra2), (dec1, dec2)]`

#### Right ascension band

In [ ]:
ztf_object_box_ra = ztf_object.box(ra=[-65, -60])
ztf_object_box_ra

In [ ]:
ztf_object_box_ra.plot_pixels("ZTF_DR14 - RA band pixel map")

#### Declination band

In [ ]:
ztf_object_box_dec = ztf_object.box(dec=[12, 15])
ztf_object_box_dec

In [ ]:
ztf_object_box_dec.plot_pixels("ZTF_DR14 - DEC band pixel map")

#### Right ascension and declination bands

In [ ]:
ztf_object_box = ztf_object.box(ra=[-65, -60], dec=[12, 15])
ztf_object_box

In [ ]:
ztf_object_box.plot_pixels("ZTF_DR14 - box pixel map")

We can stack a several number of filters, which are applied in sequence. For example, `catalog.box().polygon_search()` should result in a perfectly valid HiPSCat catalog containing the objects that match both filters.

## Previewing part of the data

Computing an entire catalog requires loading all of its resulting data into memory, which is expensive and may lead to out-of-memory issues. 

Often our goal is to have a peek at a slice of data to make sure the workflow output is reasonable (e.g. to assess if some new created columns are present and their values have been properly processed). `head()` is a pandas-like method which allows us to preview part of the data for this purpose. It iterates over the existing catalog partitions, in sequence, and finds up to `n` number of rows.

Notice that this method implicitly calls `compute()`.

In [ ]:
ztf_object_cone.head()

By default the first 5 rows of data will be shown but we can specify a higher number if we need.

In [ ]:
ztf_object_cone.head(n=10)

## Closing the Dask client

In [ ]:
client.close()